# IS-02 Machine Learning - Data and Web Science
## Lecture 10: Neural Networks
## Project 9 - Image Classifier
### <i>Avgitidis Konstantinos AM: 65</i>

In [1]:
import re
import torch
from torchvision import models, transforms
from PIL import Image

In [2]:
#Importing the pretrained ResNet
resnet = models.resnet101(pretrained=True)

In [3]:
#Preprocess for our images
preprocess = transforms.Compose([
    transforms.Resize(256,interpolation=Image.BICUBIC),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

In [4]:
#Getting ImageNet's labels
with open('labels.txt') as f:
    labels = [line.strip() for line in f.readlines()]

In [5]:
#Defining a function that
def predict(name,path,count):
    print("-".join([ "-" for _ in range(40)]),end="\n")
    print(f"{name} top 5 predictions for the {count} input images",end="\n")
    for k in range(count):
        img = Image.open(f"{path}{k+1}.jpg") #getting the image
        img_t = preprocess(img) #applying the preprocess transformations
        batch_t = torch.unsqueeze(img_t, 0) #unsqueezing the image tensor
        if torch.cuda.is_available(): #using cuda to calculate the results quicker
            batch_t = batch_t.to('cuda')
            resnet.to('cuda')

        resnet.eval() #setting our module in evaluation mode
        out = resnet(batch_t) #getting our results
        _, index = torch.max(out, 1)
        _, indices = torch.sort(out, descending=True) #sorting results by descending order
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100

        print(f"\nImage{k+1} results:",end="\n")
        #getting the top 5 predictions
        predictions = [(labels[idx], str(round(percentage[idx].item(),2)) + "%") for idx in indices[0][:5]]
        for i in predictions:
            print(re.sub('["]',"",i[0]).title(),i[1])
    print("-".join([ "-" for _ in range(40)]),end="\n")

In [6]:
#Predicting the breed of the dog in our images
predict("Ermis","Ermis/Ermis",3)
predict("Fred","Fred/Fred",3)
predict("Jake","Jake/jake",4)

-------------------------------------------------------------------------------
Ermis top 5 predictions for the 3 input images

Image1 results:
Borzoi, Russian Wolfhound 14.66%
German Shepherd, German Shepherd Dog, German Police Dog, Alsatian 11.87%
Collie 6.81%
Pembroke, Pembroke Welsh Corgi 5.59%
Cardigan, Cardigan Welsh Corgi 4.64%

Image2 results:
Cardigan, Cardigan Welsh Corgi 42.54%
German Shepherd, German Shepherd Dog, German Police Dog, Alsatian 22.84%
Pembroke, Pembroke Welsh Corgi 18.94%
Collie 5.01%
Border Collie 2.48%

Image3 results:
Patio, Terrace 25.08%
Borzoi, Russian Wolfhound 10.27%
Pembroke, Pembroke Welsh Corgi 6.99%
Eskimo Dog, Husky 6.07%
Siberian Husky 4.43%
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Fred top 5 predictions for the 3 input images

Image1 results:
English Setter 41.58%
Dalmatian, Coach Dog, Carriage Dog 22.93%
German Short-Haired Poi